## Making the DataFrame.


In [14]:
# Importing required packages.
import pandas as pd
import numpy as np

# Using the website link given and making a DataFrame our of it.
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
Neighbourhood = pd.DataFrame(dfs[0])

#Displaying DataFrame
Neighbourhood.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Let's change the string 'Not assigned' to __NaN__ so it can be removed from DataFrame.

In [15]:
#Chaning 'Not assigned' to NaN.
Neighbourhood.replace('Not assigned', np.NaN, inplace=True)

#Dropping NaN values.
Neighbourhood.dropna(inplace=True)

#Displaying final result.
Neighbourhood.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Let's see how many rows and columns we have in the end.

In [17]:
Neighbourhood.shape

(103, 3)